In [2]:
import pandas as pd

In [ ]:
# 1. URL sąrašas
urls = [
    "https://github.com/marukqs/techmuge/blob/85544144cb731577c29a76845a8e37d468c4a679/valstybiniai_numeriai_csv_part1.csv?raw=true",
    "https://github.com/marukqs/techmuge/blob/85544144cb731577c29a76845a8e37d468c4a679/valstybiniai_numeriai_csv_part2.csv?raw=true",
    "https://github.com/marukqs/techmuge/blob/85544144cb731577c29a76845a8e37d468c4a679/valstybiniai_numeriai_csv_part3.csv?raw=true",
]

# 2. Duomenų nuskaitymas ir apdorojimas
dataframes = []
for url in urls:
    # Nuskaitymas į DataFrame
    df = pd.read_csv(url, header=None, names=["numeris"])
    # Pridedamas stulpelis su failo vardu
    df["failo_vardas"] = url.split("/")[-1].split("?")[0]
    dataframes.append(df)

# 3. Sujungimas į vieną DataFrame
combined_df = pd.concat(dataframes, ignore_index=True)

# 4. Unikalių numerių su pasikartojimais skaičiavimas
numeris_counts = combined_df["numeris"].value_counts()
pasikartojantys_numeriai = numeris_counts[numeris_counts > 1]
print(f"Pasikartojančių numerių skaičius: {len(pasikartojantys_numeriai)}")

# 5. Paliekame tik unikalius numerius
unikalus_df = combined_df.drop_duplicates(subset="numeris")

# 6. Pridedame stulpelį 'tipas'
def classify_number(numeris):
    import re
    if re.fullmatch(r"[A-Z]{3}\d{3}", numeris):
        return "automobilis"
    elif re.fullmatch(r"[A-Z]{2}\d{3}", numeris):
        return "priekaba"
    elif re.fullmatch(r"\d{3}[A-Z]{2}", numeris):
        return "motociklas"
    elif re.fullmatch(r"\d{2}[A-Z]{3}", numeris):
        return "mopedas"
    elif re.fullmatch(r"[A-Z]{2}\d{2}", numeris):
        return "keturratis"
    elif re.fullmatch(r"T\d{5}", numeris, re.IGNORECASE):
        return "taksi"
    elif re.fullmatch(r"H\d{5}", numeris, re.IGNORECASE):
        return "istorinis"
    elif re.fullmatch(r"\d{4}H", numeris, re.IGNORECASE):
        return "istorinis mažasis"
    elif re.fullmatch(r"P\d{5}", numeris, re.IGNORECASE):
        return "prekyba"
    elif re.fullmatch(r"P\d{4}", numeris, re.IGNORECASE):
        return "prekyba mažasis"
    elif re.fullmatch(r"E[A-Z]\d{4}", numeris, re.IGNORECASE):
        return "EV"
    elif re.fullmatch(r"\d{6}", numeris):
        return "diplomatinis"
    elif re.fullmatch(r"\d{5}", numeris):
        return "diplomatinis mažasis"
    elif re.fullmatch(r"\d{4}[A-Z]{2}", numeris):
        return "tranzitinis"
    elif re.fullmatch(r"\d{4}[A-Z]", numeris):
        return "tranzitinis mažasis"
    else:
        return "nežinomas"

unikalus_df["tipas"] = unikalus_df["numeris"].apply(classify_number)

# 7. Pridedame stulpelį "eil_nr"
unikalus_df = unikalus_df.reset_index(drop=True)
unikalus_df["eil_nr"] = unikalus_df.index + 1

# 8. Įrašome į CSV failą
unikalus_df.to_csv("numeriai.csv", index=False)

# 9. Išvesti po du kiekvieno tipo numerius
sampled_df = unikalus_df.groupby("tipas").apply(lambda x: x.head(2)).reset_index(drop=True)
print("Po du kiekvieno tipo numerius:")
print(sampled_df)

# 10. Rezultatai
print("Pirmi 10 unikalių įrašų su tipu ir eilės numeriais:")
print(unikalus_df.head(10))


In [3]:
# 11. Papildoma analizė
# Perskaitome failą 'numeriai.csv'
read_df = pd.read_csv("numeriai.csv")

# Skaičiuojame kiekvieno tipo numerių skaičių
count_by_type = read_df["tipas"].value_counts()
print("Kiekvieno tipo numerių skaičius:")
print(count_by_type)

# Įrašome neteisingus įrašus į neteisingi_df
neteisingi_df = read_df[read_df["tipas"] == "nežinomas"]
print("Neteisingi numeriai:")
print(neteisingi_df)

# Analizuojame 5 dažniausias raides kiekvieno tipo numeriuose
def extract_letters(numeris):
    return "".join(filter(str.isalpha, numeris))

read_df["letters"] = read_df["numeris"].apply(extract_letters)
analize_df = read_df.groupby("tipas")["letters"].apply(
    lambda x: pd.Series(list("".join(x))).value_counts().head(5)
).reset_index()
analize_df.columns = ["tipas", "raidė", "dažnis"]

print("5 dažniausios raidės kiekvieno tipo numeriuose:")
print(analize_df)


Kiekvieno tipo numerių skaičius:
tipas
automobilis             2000
mopedas                  182
motociklas                99
priekaba                  88
EV                        45
diplomatinis              39
taksi                     17
prekyba                   13
istorinis                 10
keturratis                 4
istorinis mažasis          3
diplomatinis mažasis       1
Name: count, dtype: int64
Neteisingi numeriai:
Empty DataFrame
Columns: [numeris, failo_vardas, tipas, eil_nr]
Index: []
5 dažniausios raidės kiekvieno tipo numeriuose:
                tipas raidė  dažnis
0                  EV     E      49
1                  EV     Z       4
2                  EV     I       4
3                  EV     H       3
4                  EV     U       3
5         automobilis     N     284
6         automobilis     M     281
7         automobilis     C     280
8         automobilis     V     275
9         automobilis     B     274
10          istorinis     H      10
11  istorini